In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import openai as oa

# Load environment variables
_ = load_dotenv(find_dotenv())

# Set OpenAI API key
oa.api_key = os.environ['OPENAI_API_KEY']




In [2]:

# Path to the .docx file
#docx_path = '../Evaluation Sets/Raptor Contract.docx'


'''
# Load the document
doc = Document(docx_path)

# Extract text from non-empty paragraphs
docx_texts = [paragraph.text.strip() for paragraph in doc.paragraphs if paragraph.text.strip()]

# Split text into chunks based on separators
character_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=500, chunk_overlap=0)
character_split_texts = character_splitter.split_text('\n\n'.join(docx_texts))

# Further split into smaller segments based on token count
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)
token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

token_split_texts'''


# Chunk the data
#text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
#chunks = text_splitter.split_documents(documents)
#chunks

'\n# Load the document\ndoc = Document(docx_path)\n\n# Extract text from non-empty paragraphs\ndocx_texts = [paragraph.text.strip() for paragraph in doc.paragraphs if paragraph.text.strip()]\n\n# Split text into chunks based on separators\ncharacter_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=500, chunk_overlap=0)\ncharacter_split_texts = character_splitter.split_text(\'\n\n\'.join(docx_texts))\n\n# Further split into smaller segments based on token count\ntoken_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)\ntoken_split_texts = []\nfor text in character_split_texts:\n    token_split_texts += token_splitter.split_text(text)\n\ntoken_split_texts'

In [3]:
import os
from docx import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Replace with your .docx file path
docx_path = '../Evaluation Sets/Raptor Contract.docx'

# Load the document using python-docx
doc = Document(docx_path)

# Extract text from non-empty paragraphs
docx_texts = [paragraph.text.strip() for paragraph in doc.paragraphs if paragraph.text.strip()]

# Write the extracted text to a temporary file (optional)
temp_text_file = 'temp_text_file.txt'
with open(temp_text_file, 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(docx_texts))

# Load the data using TextLoader
loader = TextLoader(temp_text_file)
documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Clean up: delete temporary text file (optional)
os.remove(temp_text_file)

# Now you have `chunks` containing the segmented text data ready for further processing
print(chunks)


Created a chunk of size 501, which is longer than the specified 500
Created a chunk of size 668, which is longer than the specified 500
Created a chunk of size 548, which is longer than the specified 500
Created a chunk of size 689, which is longer than the specified 500
Created a chunk of size 1628, which is longer than the specified 500
Created a chunk of size 1025, which is longer than the specified 500
Created a chunk of size 1530, which is longer than the specified 500
Created a chunk of size 679, which is longer than the specified 500
Created a chunk of size 636, which is longer than the specified 500
Created a chunk of size 591, which is longer than the specified 500
Created a chunk of size 585, which is longer than the specified 500
Created a chunk of size 873, which is longer than the specified 500
Created a chunk of size 749, which is longer than the specified 500
Created a chunk of size 2930, which is longer than the specified 500
Created a chunk of size 515, which is longer

[Document(page_content='STOCK PURCHASE AGREEMENT\n\nBY AND AMONG\n\n[BUYER],\n\n[TARGET COMPANY],\n\nTHE SELLERS LISTED ON SCHEDULE I HERETO\n\nAND\n\nTHE SELLERS’ REPRESENTATIVE NAMED HEREIN\n\nDated as of [●]', metadata={'source': 'temp_text_file.txt'}), Document(page_content='Dated as of [●]\n\n[This document is intended solely to facilitate discussions among the parties identified herein.  Neither this document nor such discussions are intended to create, nor will either or both be deemed to create, a legally binding or enforceable offer or agreement of any type or nature, unless and until a definitive written agreement is executed and delivered by each of the parties hereto.', metadata={'source': 'temp_text_file.txt'}), Document(page_content='This document shall be kept confidential pursuant to the terms of the Confidentiality Agreement entered into by the parties and, if applicable, its affiliates with respect to the subject matter hereof.]\n\nTABLE OF CONTENTS\n\nARTICLE I DEFIN

In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

# Setup vector database
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

# Populate vector database
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()

/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


Started /home/mahbubah/.cache/weaviate-embedded: process ID 6810


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-07-06T10:58:37+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-07-06T10:58:37+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-07-06T10:58:37+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-07-06T10:58:38+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-07-06T10:58:38+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define prompt template
template = """"You are a helpful expert contract advisor assistant. 
Your users are asking questions about information contained in the contract.
You will be shown the user's question, and the relevant information from the contract. 
Answer the user's question using only this information.
        
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)



/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
# List of test questions
'''test_questions = [
    "Is there a non-compete obligation to the Advisor?",
    "Can the Advisor charge for meal time?",
    "In which street does the Advisor live?",
    "Is the Advisor entitled to social benefits?"
]

# Test each question
for question in test_questions:
    # Prepare input data
    input_data = {
        "question": question,
        "context": None  # Replace with your actual context if needed
    }

    # Execute RAG pipeline
    try:
        output = rag_chain(input_data)
        print(f"Question: {question}")
        print("Generated Answer:", output)
        print("=" * 50)
    except Exception as e:
        print(f"Error generating answer for question '{question}': {e}")'''

'test_questions = [\n    "Is there a non-compete obligation to the Advisor?",\n    "Can the Advisor charge for meal time?",\n    "In which street does the Advisor live?",\n    "Is the Advisor entitled to social benefits?"\n]\n\n# Test each question\nfor question in test_questions:\n    # Prepare input data\n    input_data = {\n        "question": question,\n        "context": None  # Replace with your actual context if needed\n    }\n\n    # Execute RAG pipeline\n    try:\n        output = rag_chain(input_data)\n        print(f"Question: {question}")\n        print("Generated Answer:", output)\n        print("=" * 50)\n    except Exception as e:\n        print(f"Error generating answer for question \'{question}\': {e}")'

In [12]:
from datasets import Dataset

# Define your questions, answers, and ground truths
questions = [
    "Is there a non-compete obligation to the Advisor?",
    "Can the Advisor charge for meal time?",
    "In which street does the Advisor live?",
    "Is the Advisor entitled to social benefits?",
]

ground_truths = [
    "Yes. During the term of engagement with the Company and for a period of 12 months thereafter.",
    "No. See Section 6.1, Billable Hour doesn’t include meals or travel time.",
    "1 Rabin st, Tel Aviv, Israel",
    "No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company.",
]

answers = []
contexts = []


for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# Create the dataset dictionary
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths  
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


for i in range(3):  # Adjust range depending on how many entries you want to view
    print(dataset[i])



/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump

{'question': 'Is there a non-compete obligation to the Advisor?', 'answer': 'Yes, there is a non-solicitation obligation for all current and former employees, consultants, and other individual service providers who have signed a confidentiality, non-solicitation, inventions assignment agreement, or other such Contractual Obligation prohibiting disclosure or use of Company or one of the Subsidiaries confidential information.', 'contexts': ['Nonsolicitation.\n\n[Reserved]', 'All current and former employees, consultants and other individual service providers have signed (i) either an offer letter, employment agreement or a consulting agreement with the applicable Acquired Company and (ii) a confidentiality, non-solicitation, inventions assignment agreement, or other such Contractual Obligation prohibiting disclosure or use of Company or one of the Subsidiaries confidential information and/or assigning all intellectual property created in the framework of their engagement with of the Comp

/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/pydantic/main.py:1086: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [13]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    raise_exceptions=False  
)

df = result.to_pandas()

# Print the dataframe to inspect the results
print(df)


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/ragas/executor.py", line 87, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 663, in run_until_complete
    self._check_running()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 622, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
/usr/lib/python3.12/threading.py:1075: RuntimeWarning: coroutine 'Runner._aresults' was never awaited
  self._invoke_excepthook(self)


ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/ipykernel/iostream.py:123: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  await self._event_pipe_gc()
